In [3]:
import pandas as pd
import numpy as np

In [186]:
# Importando CSV de treino
# Importanto CSV de validação do kaggle
# AMBOS Pré processados

df_train = pd.read_csv("../Analise_dados_titanic/X_desbalanced_overSampling.csv")
targets = pd.read_csv("../Analise_dados_titanic/df_train_Targets.csv")
df_submission = pd.read_csv("../Analise_dados_titanic/df_submission_final.csv")


In [187]:
df_train.head()

,Pclass,Fare,Sex_bin,Title_int,Parch
0,3,7.2500,0,1,0
1,1,71.2833,1,3,0
2,3,7.9250,1,2,0
3,1,53.1000,1,3,0
4,3,8.0500,0,1,0


In [188]:
df_submission.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Sex_bin,Embarked_int,Title,Title_int,Family_size
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q,0,3,Mr.,1.0,0
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S,1,1,Mrs.,3.0,1
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q,0,3,Mr.,1.0,0
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S,0,1,Mr.,1.0,0
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S,1,1,Mrs.,3.0,2


In [190]:
# Definindo as colunas que seram utilizadas para treinamento e estimativa

colunas = ['Pclass','Fare','Sex_bin','Title_int','Parch']

In [191]:
X = df_train[colunas]
try:
    y = df_train['Survived']
except:
    y = targets

envio = df_submission[colunas]

In [192]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 953 entries, 0 to 952
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Pclass     953 non-null    int64  
 1   Fare       953 non-null    float64
 2   Sex_bin    953 non-null    int64  
 3   Title_int  953 non-null    int64  
 4   Parch      953 non-null    int64  
dtypes: float64(1), int64(4)
memory usage: 37.4 KB


In [193]:
y.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 953 entries, 0 to 952
Data columns (total 1 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   Survived  953 non-null    int64
dtypes: int64(1)
memory usage: 7.6 KB


In [194]:

import os
os.environ['OMP_NUM_THREADS'] = '1'

In [195]:
# Imports do sklearn

from sklearn.model_selection import train_test_split

In [196]:
# Dividindo os dados de treino em dois grupos
X_group_train, X_group_test, y_group_train, y_group_test = train_test_split(X,y, test_size=0.3)

# =================================================================

In [88]:
# Imports do sklearn

from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectPercentile, chi2

from sklearn.model_selection import cross_val_score, StratifiedKFold

from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

# Definição do modelo
clf = RandomForestClassifier(class_weight='balanced', criterion='entropy',
                       max_depth=10, n_estimators=200, n_jobs=-1,
                       random_state=5)

In [197]:
clf = XGBClassifier(base_score=0.5, booster='gbtree',
              colsample_bylevel=0.7475082544980372, colsample_bynode=1,
              colsample_bytree=0.7303494599037559, gamma=0.2782039943046268,
              gpu_id=-1, importance_type='gain', interaction_constraints='',
              learning_rate=0.0019499298126315443, max_delta_step=0,
              max_depth=7, min_child_weight=1,
              monotone_constraints='()', n_estimators=1600, n_jobs=1,
              num_parallel_tree=1, random_state=4, reg_alpha=0.7569700751598903,
              reg_lambda=1.291937538747793, scale_pos_weight=1, seed=4,
              subsample=0.5056122040729307, tree_method='exact',
              validate_parameters=1, verbosity=None, use_label_encoder=False,
              disable_default_eval_metric=True)

## Realizando a primeira validação com todas as FEATURES

In [51]:
# Realizando Validação cruzada
for i in range(3):
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=i)
    results = cross_val_score(clf, X_group_train, y_group_train, cv= cv)

    # Calculando as metricas
    mean = results.mean()
    dvp = results.std()
    intervalo = f'{(mean - 2*dvp)*100:.3f} ~~ {(mean + 2*dvp)*100:.3f}'

    # Mostrando resultados
    print(f"""
        Acuracia: {mean:.4f}
        Intervalo de acuracia: {intervalo}
    """)


        Acuracia: 0.8306
        Intervalo de acuracia: 76.799 ~~ 89.314
    

        Acuracia: 0.8262
        Intervalo de acuracia: 74.735 ~~ 90.512
    

        Acuracia: 0.8249
        Intervalo de acuracia: 79.755 ~~ 85.218
    


## Realizando a validação com as melhores "FEATURES" definidas pelo seletor percentual

In [198]:
# Definindo seletor de FEATURES
slc_feat = SelectPercentile(chi2,percentile=100)

In [199]:
# Selecionando as melhores FEATURES para o modelo no grupo de treino para validação cruzada
features_train = slc_feat.fit_transform(X_group_train,y_group_train)

# Transformando as FEATURES do dataset de test
features_test = slc_feat.transform(X_group_test)

In [200]:
slc_feat.inverse_transform(features_train)[0]

array([ 1. , 53.1,  0. ,  1. ,  0. ])

In [201]:
features_train.shape

(667, 5)

In [202]:
# Realizando Validação cruzada
for i in range(3):
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=i)
    results = cross_val_score(clf, features_train, y_group_train, cv= cv)

    # Calculando as metricas
    mean = results.mean()
    dvp = results.std()
    intervalo = f'{(mean - 2*dvp)*100:.3f} ~~ {(mean + 2*dvp)*100:.3f}'

    # Mostrando resultados
    print(f"""
        Acuracia: {mean*100:.3f}
        Intervalo de acuracia: {intervalo}
    """)


        Acuracia: 76.299
        Intervalo de acuracia: 67.657 ~~ 84.940
    

        Acuracia: 76.764
        Intervalo de acuracia: 71.772 ~~ 81.755
    

        Acuracia: 76.613
        Intervalo de acuracia: 72.587 ~~ 80.639
    


In [203]:
# Realizando treino com subcontunto de treino com as FEATURES SELECIONADAS
clf.fit(features_train, y_group_train)

XGBClassifier(base_score=0.5, booster='gbtree',
              colsample_bylevel=0.7475082544980372, colsample_bynode=1,
              colsample_bytree=0.7303494599037559,
              disable_default_eval_metric=True, gamma=0.2782039943046268,
              gpu_id=-1, importance_type='gain', interaction_constraints='',
              learning_rate=0.0019499298126315443, max_delta_step=0,
              max_depth=7, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=1600, n_jobs=1,
              num_parallel_tree=1, random_state=4, reg_alpha=0.7569700751598903,
              reg_lambda=1.291937538747793, scale_pos_weight=1, seed=4,
              subsample=0.5056122040729307, tree_method='exact',
              use_label_encoder=False, validate_parameters=1, verbosity=None)

In [204]:
estimativas = clf.predict(features_test)

print(f"""
    {classification_report(y_group_test, estimativas)}
    {confusion_matrix(y_group_test, estimativas)}
    
Curva AUC == {roc_auc_score(y_group_test, estimativas)}
""")


                  precision    recall  f1-score   support

           0       0.79      0.89      0.84       162
           1       0.83      0.69      0.75       124

    accuracy                           0.80       286
   macro avg       0.81      0.79      0.80       286
weighted avg       0.81      0.80      0.80       286

    [[144  18]
 [ 38  86]]
    
Curva AUC == 0.7912186379928315



## APLICANDO AS FEATURES RELEVANTES

In [205]:
X_x = slc_feat.fit_transform(X,y)
X_x_sub = slc_feat.transform(envio)

In [206]:
X_x.shape

(953, 5)

# Aplicando o modelo e fazendo estimativas no conjunto de dados

In [207]:
# Com FEATURES selecionadas

clf.fit(X_x, y)

predict = clf.predict(X_x_sub)

In [208]:
last_result = pd.DataFrame({'PassengerId':df_submission['PassengerId'],'Survived':predict})
last_result.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


In [211]:
last_result.to_csv("last_result.csv", index=False)